In [17]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Cipher import AES
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA512, SHA384, SHA256, SHA, MD5
from Crypto import Random
from base64 import b64encode, b64decode
import random
import base64
import hashlib

hash = "SHA-256"
class AESCipher(object):

    # initialize the class instance and creating a key from user's password
    def __init__(self, passPhrase): 
        self.bs = AES.block_size
        print ("block size", self.bs)
        # get a 256 bit key by hashing your password
        self.key = hashlib.sha256(passPhrase.encode()).digest()
        print ("256 bit key derived from password:\n", self.key)
        print ()

    # method to perform AES encryption
    def encrypt(self, plainText):
        # the plaintext length must be a multiple of blocksize
        # Pad the data if necessary
        padded = self._pad(plainText)
        # Each time we encrypt we will need a random initialization vector
        iv = Random.new().read(AES.block_size)
        # Instantiate a cipher object from the AES library
        # Give it the key, set it to cipher-block-chaining mode, and
        # give it the initialization vector
        cipherObject = AES.new(self.key, AES.MODE_CBC, iv)
        # Encrypt the data/padding after converting to byte array
        encrypted = cipherObject.encrypt(padded.encode('utf-8'))
        # concatenate the iv and the ciphertext and convert to base64                                 
        encoded = base64.b64encode(iv + encrypted)
        return encoded

    # method to perform AES decryption
    def decrypt(self, cipherText):
        # decode the base64 string which contains the IV and ciphertext
        encrypted = base64.b64decode(cipherText) 
        # strip off the initialization vector
        iv = encrypted[:AES.block_size]
        # instantiate an AES cipher object with same parameters as before
        cipherObject = AES.new(self.key, AES.MODE_CBC, iv)
        # decrypt the string, strip off the padding and return the plaintext
        return self._unpad(cipherObject.decrypt(encrypted[AES.block_size:]))

    # method to pad a string to an even multiple of block size
    # the pad character is a hex digit indicating the length of the padding
    def _pad(self, s):
        return s + (self.bs - len(s) % self.bs) * chr(self.bs - len(s) % self.bs)

    # method to strip the padding
    @staticmethod
    def _unpad(s):
        return s[:-ord(s[len(s)-1:])]
    
def newkeys(keysize):
   random_generator = Random.new().read
   key = RSA.generate(keysize, random_generator)
   private, public = key, key.publickey()
   return public, private

def importKey(externKey):
   return RSA.importKey(externKey)

def getpublickey(priv_key):
   return priv_key.publickey()

def encrypt(message, pub_key):
   cipher = PKCS1_OAEP.new(pub_key)
   return cipher.encrypt(message)

def decrypt(ciphertext, priv_key):
   cipher = PKCS1_OAEP.new(priv_key)
   return cipher.decrypt(ciphertext)

def sign(message, priv_key, hashAlg = "SHA-256"):
   global hash
   hash = hashAlg
   signer = PKCS1_v1_5.new(priv_key)
   
   if (hash == "SHA-512"):
      digest = SHA512.new()
   elif (hash == "SHA-384"):
      digest = SHA384.new()
   elif (hash == "SHA-256"):
      digest = SHA256.new()
   elif (hash == "SHA-1"):
      digest = SHA.new()
   else:
      digest = MD5.new()        
   digest.update(message)
   return signer.sign(digest)

def verify(message, signature, pub_key):
   signer = PKCS1_v1_5.new(pub_key)
   if (hash == "SHA-512"):
      digest = SHA512.new()
   elif (hash == "SHA-384"):
      digest = SHA384.new()
   elif (hash == "SHA-256"):
      digest = SHA256.new()
   elif (hash == "SHA-1"):
      digest = SHA.new()
   else:
      digest = MD5.new()
   digest.update(message)
   return signer.verify(digest, signature)

BS = 16
pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS) 
unpad = lambda s : s[0:-ord(s[-1])]

#
#    The rest is up to you....  make sure to print out appropriate values as necessary
#
#### Generate keypairs for Bob and Alice
#### Generate keypairs for Bob and Alice
Bob_publicKey, Bob_privateKey = newkeys(2048)
Alice_publicKey, Alice_privateKey = newkeys(2048)

#Alice tasks
#### Hash a pass phrase to create a 256 bit (32 byte) session key
digest = SHA256.new()
digest.update(b"Yibo Xu")
sessionKey = digest.digest()
print("The session key Alice have to send: \n",sessionKey)
print ()
print ()
#### Generate a public and private keypair

#### encrypt the session key with Bob's public key, convert to base64 and send the encoded, encrypted key to Bob
encrypted_msg = encrypt(sessionKey, Bob_publicKey)
print ("encrypted session key: \n", encrypted_msg)
print ()
encoded_encrypted_msg = b64encode(encrypted_msg)#convert to base64

#print(priv.exportKey(format='PEM', passphrase="Yibo Xu"))
#print ()
#with open("cs356key.pem","wb") as fd:
    #fd.write(priv.exportKey(format="PEM"))

#### EXTRA CREDIT:  Use AES and the session key to encrypt a long message to Bob
Alice_msg = "Bob, I love you. mailman:Yibo Xu"
print("This is the message have to send: ",Alice_msg)
print()
myCipher = AESCipher(str(sessionKey))
cipherText = myCipher.encrypt(Alice_msg)
#print ("The base64 encoding of the IV and cipherText:\n", cipherText)
#print ()

#with open("lab3Cipher.txt", "wb") as fd:  # note the use of "wb"
    #fd.write(cipherText)
    
#     The message should give a good "soap opera" ending to our story of love and encryption

#### EXTRA CREDIT:  Create a digital signature for the AES-encrypted file
signature = sign(cipherText, Alice_privateKey)
encoded_signature = b64encode(signature)
print ()
print ("this is the base64 version of the signature that would be sent in an email:\n", encoded_signature)
print()
#### "send the file to Bob" (basically do nothing, the info will be stored in program variables.

#msg = b"Bob, I'm not good enough for you.  You deserve someone better.  Someone like Eve!"
#Bob tasks
#### EXTRA CREDIT:  verify the message is intact and came from Alice
#msg = Alice_msg.encode('utf-8')
verification = verify(cipherText, b64decode(encoded_signature), Alice_publicKey)
print ("\nThe verification proved to be", verification)
print()

#### decrypt the session key and decode it from base64
decoded_encrypted_msg = b64decode(encoded_encrypted_msg)
print ("decoded from base64:\n", decoded_encrypted_msg)
print ()
decrypted_sessionkey = decrypt(decoded_encrypted_msg, Bob_privateKey)
print ("The session key that Bob received:\n", decrypted_sessionkey)
print ()

#assert decrypted_msg == sessionKey
#### EXTRA CREDIT: use the session key to decrypt Alice's message
#with open("lab3Cipher.txt","r") as fd:
    #cipherText = fd.read()
myCipher1 = AESCipher(str(decrypted_sessionkey))

# decrypt the file

plainText1 = myCipher1.decrypt(cipherText)
print ("The Alice's message:\n",plainText1)



The session key Alice have to send: 
 b'\x1e\xb4W\x97k\x11E\xb4\xa8\xdc\xc2\x8e\xca\xb9\x93\xd7u\xf7\x8a\xa7\xa2u,C,\x07mR\x14\xd0\xca\x88'


encrypted session key: 
 b'k\xda>\xefVif\xcc\xdc<\x10\x93;\x8b$\x9f\x19\xf6%\xf8\xaf\x8f\xfa\xa3e\xfc\x95\xd9"\xba]\x94\x01\x9e\xb0\xcaf@*\xe3\xc2\xcbW4z$\xf9\xfb\xd3\x9b)\xc7\x93d\xb0\x92\x99\xfd_\xc1O@\xb2u\x9a\xeeI\xfe\x12\x80\xa7k6\xd6\x96Ib\xda\xcb\x16g\xcd\xaa\x00\xfaij\x0cU\x8e\x02\x9ew\xe1D\xa5\x0e\x0b\x0b\xe2\xaf\x9c\xff\x15\xa2\xb7\x00\x01\xde\xb0_h\x19\x1e\xdb!\xc4gT\x86\xa1\xfb\xcb#$ P\xd4-n\xbf\x0c{\xf1\xee\x9fa)\x01\x05u\xc6\x1c\x9a\xabzA\xbbF\xde\t\xea-\x85\x05\nF+\xe6\x1d7.{\x89\xf4\xa6\xc0\xb0\x1b\t\xcf\x1ca\xec\x8aX\xe5\x11)\xdd-\x06t5\x86\x19\xbd\xe6\xc3\xd1B\x1fup*\xed\xd6\xb8\xad\xfc\x14\x12\x16W\xd7W+\xae\xe7L\xc6\xb9\xe3\xe0F\xde\x05\xae*X\x94\'\xc4\xda\xa0U\x94\x89)\x92GY\x0c\r\xe70|\xcf\x0f\xfd\x89!i\x9b\x97_\x07\x8c\x11\x8f\x06E\xed\xf5D\xe6'

This is the message have to send:  Bob, I love you. mailman:Yibo Xu

block siz